In [1]:
import numpy as np
from graphviz import Digraph

# STEP1: Displaying MDP graph

In [2]:
mdp_graph = Digraph('MDP', filename='mdp.gv')

In [3]:
transitions_states = {
    's0': {
        'a0': {'s0': 0.5, 's2': 0.5},
        'a1': {'s2': 1.0}
    },
    's1': {
        'a0': {'s0': 0.7, 's1': 0.1, 's2': 0.2},
        'a1': {'s1': 0.95, 's2': 0.05}
    },
    's2': {
        'a0': {'s0': 0.4, 's2': 0.6},
        'a1': {'s0': 0.3, 's1': 0.3, 's2': 0.4}
    }
}

In [4]:
rewards = {
    ('s2', 'a1', 's0'): -1,
    ('s1', 'a0', 's0'): +5
}

In [5]:
actions = {'a0', 'a1'}

In [6]:
# Create a new Digraph object
dot = Digraph(comment='MDP')

In [7]:
# Add nodes to the graph
for state in transitions_states:
    dot.node(state)

In [8]:
# Add edges to the graph
for state in transitions_states:
    for action in actions:
        for next_state, probability in transitions_states[state][action].items():
            dot.edge(state, next_state, label='(' + action + ' - ' + str(probability) + ')')

In [9]:
# Add rewards to the graph
for key, value in rewards.items():
    dot.edge(key[0], key[2], label=str(value))

In [10]:
# Render the graph
dot.render('graph', view=True)

'graph.pdf'

# STEP 2: Solving MDP using Value Iteration

In [11]:
max_iterations=100
gamma = 0.9
delta = 1e-6

In [12]:
V = {'s0': 0, 's1': 0, 's2': 0}

In [13]:
# Value iteration loop
for i in range(max_iterations):
    delta_i = 0
    # Compute the state-action values Q_i(s,a)
    Q = {}
    for s in transitions_states:
        for a in transitions_states[s]:
            q = 0
            for s1 in transitions_states[s][a]:
                prob = transitions_states[s][a][s1]
                if (s, a, s1) in rewards:
                    r = rewards[(s, a, s1)]
                else:
                    r = 0
                q += prob*(r + gamma*V[s1])
            Q[(s, a)] = q
    # Update the value function V_i+1(s)
    for s in transitions_states:
        v = V[s]
        V[s] = max([Q[(s, a)] for a in transitions_states[s]])
        delta_i = max(delta_i, abs(v - V[s]))
    # Check for convergence
    if delta_i < delta:
        break

In [15]:
# Find the optimal policy
pi = {}
for s in transitions_states:
    pi[s] = max(transitions_states[s], key=lambda a: sum(transitions_states[s][a][s1]*(rewards.get((s, a, s1), 0) + gamma*V[s1]) for s1 in transitions_states[s][a]))

In [16]:
# Compute the average reward
avg_reward = sum(sum(transitions_states[s][pi[s]][s1]*(rewards.get((s, pi[s], s1), 0)) for s1 in transitions_states[s][pi[s]]) for s in transitions_states) / sum(len(transitions_states[s]) for s in transitions_states)

In [17]:
print("Optimal value function:", V)
print("Optimal policy:", pi)
print("Average reward:", avg_reward)

Optimal value function: {'s0': 3.789830037441072, 's1': 7.30280158776066, 's2': 4.21093543912048}
Optimal policy: {'s0': 'a1', 's1': 'a0', 's2': 'a1'}
Average reward: 0.5333333333333333
